In [ ]:
import nltk
import json
import os
import os.path
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Ingest JSON Data

In [ ]:
# Set Current WD to json data
if (os.getcwd()[-31:] != "Messenger-Mining\\data\\live\\json"):
    os.chdir("../data/live/json")
# Store filenames of all files in the directory to files.
files = []
for path in os.listdir(os.getcwd()):
    # check if current path is a file
    if os.path.isfile(os.path.join(os.getcwd(), path)):
        files.append(path)

participants = []
for file_name in files:
    with open(file_name,"r") as read_file:
        data = json.load(read_file)
        if len(data['participants']) <= 2:
            participants.append(data['participants'])

participants    

# Train the model

In [ ]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()


posts_text = [post.text for post in posts]

#divide train and test in 80 20
train_text = posts_text[:int(len(posts_text)*0.8)]
test_text = posts_text[int(len(posts_text)*0.2):]

#Get TFIDF features
vectorizer = TfidfVectorizer(ngram_range=(1,3), 
                             min_df=0.001, 
                             max_df=0.7, 
                             analyzer='word')

X_train = vectorizer.fit_transform(train_text)
X_test = vectorizer.transform(test_text)

y = [post.get('class') for post in posts]

y_train = y[:int(len(posts_text)*0.8)]
y_test = y[int(len(posts_text)*0.2):]

In [ ]:
# Fitting Gradient Boosting classifier to the Training set
#Can be improved with Cross Validation
gb = GradientBoostingClassifier(n_estimators = 400, random_state=0)
gb.fit(X_train, y_train)

In [ ]:
predictions_rf = gb.predict(X_test)
print(classification_report(y_test, predictions_rf))